
# EDA — Breast Cancer Wisconsin (Diagnostic)

Este notebook realiza a **Análise Exploratória de Dados (EDA)** do dataset **WDBC**, base do nosso projeto de classificação (maligno vs. benigno).

**Objetivos do EDA:**
- Entender estrutura e qualidade do dado (tipos, nulos, estatísticas).
- Explorar distribuições e possíveis outliers nas features.
- Verificar correlações entre variáveis e com a variável alvo.
- Registrar insights para orientar o pré-processamento e a modelagem.


In [1]:

# Imports básicos
import os, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

# Configurações do matplotlib
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.grid"] = True

DATA_CSV = Path("data/raw/wdbc.csv")


## 1) Carregamento do dataset

In [2]:

# Tenta ler o CSV do repositório; se não existir, baixa do scikit-learn e salva para reprodutibilidade
if DATA_CSV.exists():
    df = pd.read_csv(DATA_CSV)
    print(f"Lido de {DATA_CSV.resolve()}")
else:
    from sklearn.datasets import load_breast_cancer
    data = load_breast_cancer(as_frame=True)
    df = data.frame
    # cria pastas e salva
    DATA_CSV.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(DATA_CSV, index=False)
    print("CSV não encontrado; carregado via scikit-learn e salvo em", DATA_CSV.resolve())

# Mostra as 5 primeiras linhas
df.head()


CSV não encontrado; carregado via scikit-learn e salvo em /Users/njunior/workspace/repos/fiap/fiap-techchalenge-f3/notebooks/data/raw/wdbc.csv


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


## 2) Visão geral e qualidade do dado

In [ ]:

print("Shape:", df.shape)
print("\nTipos de dados:")
print(df.dtypes.value_counts())
print("\nValores nulos por coluna (top 10):")
print(df.isna().sum().sort_values(ascending=False).head(10))


In [ ]:

# Estatísticas descritivas das features numéricas
desc = df.describe()
desc


## 3) Distribuição da variável alvo

In [ ]:

# Convenção do WDBC: target = 0 (malignant), 1 (benign)
target_counts = df["target"].value_counts().sort_index()
print(target_counts)
print("\nProporções:")
print((target_counts / target_counts.sum()).round(3))

fig = plt.figure()
target_counts.plot(kind="bar")
plt.title("Distribuição da variável alvo (0=malignant, 1=benign)")
plt.xlabel("Classe")
plt.ylabel("Contagem")
plt.show()


## 4) Distribuições das features (histogramas)

In [ ]:

feature_cols = [c for c in df.columns if c != "target"]

# Gera um histograma por feature (cada gráfico em sua própria figura)
for col in feature_cols:
    fig = plt.figure()
    df[col].hist(bins=30)
    plt.title(f"Histograma — {col}")
    plt.xlabel(col)
    plt.ylabel("Frequência")
    plt.show()


## 5) Outliers (boxplots por feature)

In [ ]:

for col in feature_cols:
    fig = plt.figure()
    plt.boxplot(df[col].dropna(), vert=True, labels=[col])
    plt.title(f"Boxplot — {col}")
    plt.ylabel(col)
    plt.show()


## 6) Correlação entre variáveis

In [ ]:

corr = df[feature_cols].corr(method="pearson")

# Salva matriz de correlação para consulta posterior
reports_dir = Path("reports")
reports_dir.mkdir(exist_ok=True, parents=True)
corr.to_csv(reports_dir / "correlation_matrix.csv")

# Heatmap simples com matplotlib (sem seaborn)
fig = plt.figure(figsize=(10, 8))
im = plt.imshow(corr.values, cmap="viridis", aspect="auto")
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title("Matriz de correlação (features)")
plt.xticks(ticks=range(len(feature_cols)), labels=feature_cols, rotation=90)
plt.yticks(ticks=range(len(feature_cols)), labels=feature_cols)
plt.tight_layout()
plt.show()

# Correlação com o alvo (usando point-biserial ~ Pearson entre numérico e binário)
corr_with_target = df[feature_cols].corrwith(df["target"]).sort_values(ascending=False)
corr_with_target.head(10)



## 7) Insights e próximos passos

- **Balanceamento de classes**: verifique a proporção de 0/1 na célula anterior para decidir se é necessário ajustar `class_weight` (no nosso treino já usamos isso onde faz sentido).
- **Escalas**: as features estão em escalas diferentes; por isso usamos **padronização** no pipeline.
- **Correlação**: grupos de features podem estar altamente correlacionados (potencial multicolinearidade). Eventualmente pode-se testar **PCA** ou selecionar subconjuntos.
- **Outliers**: verificados nos boxplots; modelos baseados em distância podem ser sensíveis (outra razão para escalonamento).
- **Próximo**: voltar ao `train.py` para validar modelos (LogReg, Árvore, RF, SVC) e confirmar o campeão.
